In [3]:
import pandas as pd
import glob

import en_core_web_sm
from collections import Counter

import re
import swifter
# from textblob import TextBlob

import sys, os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import networkx as nx

In [2]:
files = glob.glob('./incoPat_225509/*.xls')

dfs = []
for f in files:
    try:
        dfs.append(pd.read_excel(f))
    except Exception as e:
        print("{0}: {1}".format(f, e))

df = pd.concat(dfs)

In [4]:
df.columns

Index(['序号', '标题(中)', '摘要(中)', '公开(公告)号', ' 最早公开（公告）日', '申请号', '最早申请日', '同族国家',
       '族成员个数', '扩展同族ID', '链接到incoPat', '标题(英)', '摘要(英)', '合享价值度', '优先权信息',
       '专利有效性', 'IPC主分类', 'IPC', '国民经济分类', '申请人', '标准申请人', '申请人国别代码',
       '申请人省市代码', '发明人', '被引证专利', '引证专利', '引证申请人', '被引证申请人', '被引证次数', '转让人',
       '受让人', '转让执行日', '当前被许可人'],
      dtype='object')

In [14]:
df = df.reset_index(drop=True)

In [50]:
s1 = df['被引证专利'].str.strip().str.split(';').explode().to_frame()
s1['t_index'] = s1.index

In [183]:
# df.pivot_table(columns=['公开(公告)号'], aggfunc=[len], fill_value=0)
df.loc[23071]

序号                                                       103072
标题(中)         使用用于监测、分析和控制的一体式自动化系统来提供加热、冷却、发电和能量储存的基于混合式三联产...
摘要(中)         一种流动涡流扰流器(100)，包括：油槽(32)，其中加压空气流(90)围绕油槽移动穿过槽加...
公开(公告)号       US9572897B2; JP2017518254A; US20160193299A1; U...
 最早公开（公告）日                                  2010-10-24 00:00:00
申请号           US61618961; JP2015504587; CA2868429; US1439010...
最早申请日                                       2009-04-24 00:00:00
同族国家          [中国香港, 中国台湾, 俄罗斯, 日本, 克罗地亚, 新西兰, 巴西, 新加坡, 斯洛文尼...
族成员个数                                                       603
扩展同族ID                                                140516972
链接到incoPat                                                    0
标题(英)         AUTOMATED HYBRID AQUAPONICS AND BIOREACTOR SYS...
摘要(英)         A carpule for a dental syringe provides an int...
合享价值度                                                        10
优先权信息         US61737168 20121214; US61681720 20120810; US61...
专利有效性                                   

In [51]:
s2 = df['公开(公告)号'].str.strip().str.split(';').explode().to_frame()
s2['s_index'] = s2.index

In [143]:
eds = s1.merge(s2,left_on='被引证专利', right_on='公开(公告)号')

In [209]:
refs = eds[['t_index', 's_index']].groupby('s_index').count()

In [210]:
refs_df = refs[ refs.t_index > 1000]

In [211]:
refs_df

,t_index
s_index,
11359,1021
13629,1226
16411,1385
17047,1116
22103,1120
...,...
214665,1230
215726,1899
218023,1412


In [212]:
eds_df = eds[eds.s_index.isin(refs_df.index)]

In [216]:
G = nx.from_pandas_edgelist(df=eds_df[['t_index', 's_index']], source='s_index', target='t_index', create_using=nx.DiGraph)

In [ ]:
nx.draw(G)

In [218]:
nx.write_graphml(G, './g1.graphml')

In [83]:
G1000 = nx.from_pandas_edgelist(df=eds[['t_index', 's_index']].head(1000), source='s_index', target='t_index')
nx.write_graphml(G1000, './g1000.graphml')

In [112]:
# largest_cc = max(nx.connected_components(G), key=len)

In [ ]:
# kcs = nx.k_components(G)